In [1]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.8 MB/s eta 0:00:00


In [2]:
from Crypto.Cipher import AES

#3DES 암호키와 초기화 벡터 생성에 활용
from Crypto.Hash import SHA256 as SHA

#유니코드 문자열 다루는데 쓰임
import codecs

In [6]:
class myAES():

  #3DES 객체 생성시 사용할 키와 초기화 벡터 구함
  def __init__(self, keytext, ivtext):
    '''
    kettext : 3DES 암호키 생성을 위한 문자열. 16바이트 길이.
    ivtext : 초기화 벡터 위한 문자열
    '''
    hash = SHA.new() #SHA256객체 만들고 Hash 할당
    hash.update(keytext.encode('utf-8')) #주의 SHA256.update()는 유니코드 문자열을 인자로 받지 않음. python3에서 모든 문자열은 유니코드임.
    key = hash.digest() #해시 값을 추출하여 변수 key에 할당(SHA256은 256비트 해시 생성하므로, key는 256비트(32바이트)이다.)
    self.key = key[:16]

    # 블록 암호 CBC 모드로 암호하기 위해 초기화 벡터가 필요하다.
    hash.update(ivtext.encode('utf-8')) # 초기화 벡터를 위한 해시를 갱신
    iv = hash.digest() # 해시를 얻은 후 변수 iv에 할당
    self.iv = iv[:16] #16바이트를 슬라이싱하여 초기화 벡터값으로 할당

  #단문 메시지 길이에 상관없이 암호화 가능하도록 메시지 길이를 16바이트 배수로 만들고, 이에 대한 헤더정보를 포함하여 암호화
  def makeEnabled(self, plaintext):
    fillersize = 0
    textsize = len(plaintext)
    if textsize%16 != 0:
      fillersize = 16-textsize%16

    filler = '0'*fillersize
    header = '%d' %(fillersize)
    gap = 16-len(header)
    header += '#'*gap

    return header+plaintext+filler

  #암호화
  def enc(self, plaintext):
    plaintext = make8String(plaintext)
    aes = AES.new(self.key, AES.MODE_CBC, self.iv) # 인자는 암호키, 운영모드, 초기화 벡터  (ECB, CTR 모드는 초기화 벡터가 필요없다.)
    encmsg = aes.encrypt(plaintext.encode()) # 암호화 수행
    return encmsg

  #복호화
  def dec(self, ciphertext):
    aes = AES.new(self.key, AES.MODE_CBC, self.iv)
    decmsg = aes.decrypt(ciphertext) # 복호화 수행
    return decmsg

def make8String(msg):
  msglen = len(msg)
  filler = ''
  if msglen%8 != 0:
    filler = '0'*(8-msglen%8)
  msg += filler
  return msg

def main():
  keytext = 'samsjang'
  iv = '1234'
  #msg = 'python3x' # 암호화할 메시지. 메시지 길이는 8바이트의 배여여야 한다. 아니면 오류발생. -> 문자열이 8바이트 배수가 아니라도 오류없이 암호화/복호화 가능하게 코드 수정 >> make8String 함수
  msg = 'python35ab'

  myCipher = myAES(keytext, iv)
  ciphered = myCipher.enc(msg)
  deciphered = myCipher.dec(ciphered)
  print('ORIGINAL:\t%s' %msg)
  print('CIPHERED:\t%s' %codecs.encode(ciphered, 'hex_codec'))
  print('DECIPHERED:\t%s' %deciphered)

if __name__=='__main__':
  main()

ORIGINAL:	python35ab
CIPHERED:	b'679809d47ac850213f5a996a49a0a627'
DECIPHERED:	b'python35ab000000'
